In [1]:
import requests

def search_wikipedia_current(query: str) -> str:
    """
    Performs a search using the Wikipedia API and returns the summary of the top result.
    This allows the agent to discover factual information based on the search query,
    without assuming the answer in advance.
    """
    try:
        # Step 1: Search for relevant Wikipedia pages
        search_url = "https://en.wikipedia.org/w/api.php"
        params = {
            "action": "query",
            "list": "search",
            "srsearch": query,
            "format": "json"
        }
        response = requests.get(search_url, params=params, timeout=10)
        print(response,"\n")
        response.raise_for_status()
        data = response.json()
        print(data,"\n")

        search_results = data.get("query", {}).get("search", [])
        if not search_results:
            return "No relevant results found."

        # Step 2: Get the title of the first result
        page_title = search_results[0]["title"]

        # Step 3: Retrieve the summary of the selected Wikipedia page
        summary_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{page_title.replace(' ', '_')}"
        summary_response = requests.get(summary_url, timeout=10)
        summary_response.raise_for_status()
        summary_data = summary_response.json()

        return summary_data.get("extract", "No summary found.")
    
    except Exception as e:
        return f"Search error: {e}"
result = search_wikipedia_current("Current president of France")
print(result)

Search error: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&list=search&srsearch=Current+president+of+France&format=json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))


In [2]:
import requests
import re

def extract_capitalized_pairs(text: str) -> list:
    """
    Extrae pares de palabras consecutivas que empiezan con mayúscula.
    """
    pattern = r"\b([A-Z][a-z]+)\s([A-Z][a-z]+)\b"
    matches = re.findall(pattern, text)
    return [" ".join(pair) for pair in matches]

import requests
import re
from html import unescape

def get_clean_snippets_text(country: str) -> str:
    try:
        search_url = "https://en.wikipedia.org/w/api.php"
        query = f"President of {country}"
        params = {
            "action": "query",
            "list": "search",
            "srsearch": query,
            "format": "json"
        }

        response = requests.get(search_url, params=params, timeout=10)
        response.raise_for_status()
        search_data = response.json()

        # Extraer todos los snippets
        results = search_data.get("query", {}).get("search", [])
        all_snippets = " ".join(entry.get("snippet", "") for entry in results)

        # Quitar etiquetas HTML como <span class="..."> y </span>
        clean_text = re.sub(r"<.*?>", "", all_snippets)

        # Convertir entidades HTML (como &quot;) a texto normal
        clean_text = unescape(clean_text)

        return clean_text.strip()

    except Exception as e:
        return f"Error: {e}"

# Ejemplo de uso
print(get_clean_snippets_text("Argentina"))

Error: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&list=search&srsearch=President+of+Argentina&format=json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))


In [6]:
def get_summary_of(person: str) -> str:
    """
    Returns a short summary of a famous person from Wikipedia.
    """
    try:
        title = person.replace(" ", "_")
        url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
        response = requests.get(url, timeout=10,verify=False)
        response.raise_for_status()
        data = response.json()

        return data.get("extract", "No summary available.")
    
    except Exception as e:
        return f"Error: {e}"
print(get_summary_of("Angela Merkel"))
print(get_summary_of("Shakira"))
print(get_summary_of("Barack Obama"))


c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Angela Dorothea Merkel is a German retired politician who served as Chancellor of Germany from 2005 to 2021. She is the only woman to have held the office. She was Leader of the Opposition from 2002 to 2005 and Leader of the Christian Democratic Union (CDU) from 2000 to 2018.


c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Shakira Isabel Mebarak Ripoll, known mononymously as Shakira, is a Colombian singer-songwriter. She has had a significant impact on the musical landscape of Latin America and has been credited with popularizing Hispanophone music on a global level. The recipient of various accolades, she has won four Grammy Awards and fifteen Latin Grammy Awards, including three Song of the Year wins.
Barack Hussein Obama II is an American politician who was the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African American president in American history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.


c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
import requests

def get_current_president_of(country: str) -> str:
    """
    Returns the name of the current president of a given country using Wikidata.
    """
    try:
        # Step 1: Get Wikidata ID for the country
        search_url = "https://www.wikidata.org/w/api.php"
        search_params = {
            "action": "wbsearchentities",
            "search": country,
            "language": "en",
            "format": "json",
            "type": "item"
        }
        search_resp = requests.get(search_url, params=search_params, timeout=10,verify=False)
        search_resp.raise_for_status()
        search_data = search_resp.json()
        entity_id = search_data["search"][0]["id"]

        # Step 2: Query president (P35) of the country entity
        sparql_url = "https://query.wikidata.org/sparql"
        query = f"""
        SELECT ?presidentLabel WHERE {{
          wd:{entity_id} wdt:P35 ?president.
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
        headers = {"Accept": "application/sparql-results+json"}
        sparql_resp = requests.get(sparql_url, params={"query": query}, headers=headers, timeout=10,verify=False)
        sparql_resp.raise_for_status()
        result = sparql_resp.json()
        bindings = result["results"]["bindings"]

        if not bindings:
            return "President not found."
        return bindings[0]["presidentLabel"]["value"]

    except Exception as e:  
        return f"Error: {e}"


In [14]:
print(get_current_president_of("Argentina"))  # Javier Milei
print(get_current_president_of("France"))     # Emmanuel Macron
print(get_current_president_of("Germany"))    # Frank-Walter Steinmeier


c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Javier Milei


c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Emmanuel Macron


c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Frank-Walter Steinmeier


c:\Users\eseguzkijo\AppData\Local\miniconda3\envs\TFM2\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
